# Automate Tech Blog

### Cycle of LLM Chat Model

- Understanding the business Statement
- creating data collection
- creating vectorestore
- add collection to vectorstore
- Using RAG to retrieve data from Database

# Business statement
- when developing a technical product, writing techinical blog post can help users to get started by following post and seeing how to set. now the problem is how can we ensure the post reach to the community?

- In open source, a common pratice is to retrieve data from webpage in combination of code and convert to text.

# Problem statement
 How can we leverage open source tools to do outreach with the community?

  - One way is to spread the word and connect with community is through social media, like twitter, Linkedin.
  - With the rise of Artificial Intellengence, we can automate the process of generating blogs that are enticing and will encourage users to try a tech product.


# Data Collection
 - Source - https://numpy.org/doc/

Importing necessary libraries , Langchain, OpenAI, OS

In [22]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Qdrant
from qdrant_client import models, QdrantClient
import qdrant_client

We want to use OpenAIEmbeddings so we have to get the OpenAI API Key.

In [ ]:
import os

os.environ["OPENAI_API_KEY"]

# Create Embeddings using Qdrant Vectorstore
Qdrant is a vector database & vector similarity search engine

 - First we want to create a Qdrant vector store and seed it with some data.

In [16]:
# create qdrant client

os.environ['QDRANT_HOST']
os.environ["QDRANT_API_KEY"]

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [6]:
# read the file
file="numpy.txt"
data=""

with open(file,'r') as f:
    data = f.read()

In [10]:
# split the text into chunks
#create a function to return chunks
def get_chunks(text):
    text_splitter=CharacterTextSplitter(
        separator= "\n",
        chunk_size=700,
        chunk_overlap=100,
         # second chunk start  character from 800, overlap is used to stop loosing chunk 
        length_function=len
    )

    chunks=text_splitter.split_text(text)
    return chunks

In [11]:
# get the chunks for the data
texts=get_chunks(data) 

In [12]:
len(texts)

7

In [13]:
len(data)

3606

In [20]:
# creating a new  collection and naming it.

vectors_config=models.VectorParams(
    # depends on model, we can google dimension. 1536 for openai
    # we are using openai embedding, for that size is 1536
    size=1536,
    distance=models.Distance.COSINE)

client.create_collection(
    collection_name="numpy",
    vectors_config=vectors_config,
)

True

In [27]:
# if we want to use any other embedding, we need to change size

os.environ["OPENAI_API_KEY"]

embeddings = OpenAIEmbeddings()

vector_store = Qdrant(
    client=client,
    collection_name="numpy",
    embeddings=embeddings,
)

e:\automate-tech-post\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [28]:
# add chunks to vector store
vector_store.add_texts(texts)

['6ce0d3327a174d29b01bd4b49bfdfa61',
 'ce5b689de17746cdbf5c1a2336286cee',
 '77a0b30fdd5c428f840c9b0f51436c53',
 '5e9ef869acb14293a66583c855e26f05',
 '41208f494f484ca0b98a65ac610368fb',
 'cb08df9e8c2f490e9beabbc3314875b5',
 '81ebc32802fa447d8741224bff0f86b6']

# Creating our RAG retriever
The next step is to perform a relevancy search. The user query is converted to a vector representation and matched with the vector databases.

In [32]:
# let's try passing a question through to a these vectorstore objects

from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

retriever = vector_store.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()
     

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
   

In [ ]:
# using LCEL

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Summary to generate a blog about

This blog  is a tutorial about using NumPy to solve static equilibrium problems in three-dimensional space. Readers will learn how to represent points, vectors, and moments with NumPy, find the normal of vectors, and use NumPy for matrix calculations. The tutorial covers the application of Newton's second law to simple examples of force
vectors and introduces more complex cases involving reaction forces and moments. The post also discusses the use of NumPy functions in more varied problems, including kinetic problems and different dimensions, here are some few questions we can ask our model to generate the response;

 - how to train a simple feed-forward neural network from scratch using NumPy to classify handwritten MNIST digits?
 - what is concept of masked arrays in NumPy and their usefulness in handling missing or invalid data?
 - Linear algebra on n-dimensional arrays
 - NumPy to solve static equilibrium problems in three-dimensional space


In [34]:
chain.invoke("how to train a simple feed-forward neural network from scratch using NumPy to classify handwritten MNIST digits?")


"To train a simple feed-forward neural network from scratch using NumPy to classify handwritten MNIST digits, you need to follow these steps:\n\n1. Load the MNIST dataset, which contains 60,000 training images and 10,000 test images, along with their corresponding labels.\n2. Split the training images into a training set and a validation set.\n3. Initialize the weights of the neural network.\n4. Define the activation functions and their derivatives that will be used in the network.\n5. Implement a function for forward pass and backward pass in the network.\n6. Train the model in batches using stochastic gradient descent (SGD), updating the weights based on the gradients computed during backpropagation.\n7. Test the model's accuracy on the validation set.\n8. Finally, predict the classes of the images in the test set and print the accuracy of the model's predictions.\n\nBy following these steps, you can build and train a simple feed-forward neural network from scratch using NumPy to cla

In [38]:
for chunk in chain.stream("what is concept of masked arrays in NumPy and their usefulness in handling missing or invalid data?"):
    print(chunk, end="", flush=True) 

The concept of masked arrays in NumPy is the combination of a standard numpy.ndarray and a mask. A mask is either nomask, indicating that no value of the associated array is invalid, or an array of booleans that determines for each element of the associated array whether the value is valid or not. Masked arrays are useful in handling missing or invalid data.

In [39]:
for chunk in chain.stream("Linear algebra on n-dimensional arrays"):
    print(chunk, end="", flush=True) 

Linear algebra is an important topic in the data science domain. It is particularly relevant when working with n-dimensional arrays, which are collections of data represented with numbers. Linear algebra allows for efficient computations on these arrays, which is especially important in machine learning and deep learning models that require large amounts of data. By using vectorized or matrix operations, the performance and accuracy of these models can be improved.

Conclusion